# Neural style transfer

The key notion behind implementing style transfer is the same idea that’s central
to all deep-learning algorithms: you define a loss function to specify what you want to
achieve, and you minimize this loss. You know what you want to achieve: conserving
the content of the original image while adopting the style of the reference image. If
we were able to mathematically define content and style, then an appropriate loss func-
tion to minimize would be the following:


`loss = distance(style(reference_image) - style(generated_image)) + distance(content(original_image) - content(generated_image))`

卷积网络：底层表达局部信息，高层表达整体信息。因此卷积的内容是否相似，可以从高层中计算得到；风格是否相似，可以从底层中计算得到。